In [5]:
import oracledb
import pandas as pd

connection = oracledb.connect(user='####', password='######', dsn='192.168.202.75:1521/icubeprd')



In [10]:
query1 = """
WITH cte AS (
    SELECT 
        NATIONALID,
        CASE 
            WHEN REGEXP_LIKE(DISBURSMENTDATE, '^\d{2}-\d{2}-\d{4}$') 
                THEN TO_DATE(DISBURSMENTDATE, 'DD-MM-YYYY') 
            ELSE NULL 
        END AS DISBURSMENTDATE,
        CASE 
            WHEN REGEXP_LIKE(PRINCIPALAMOUNT, '^\d+(\.\d+)?$') 
                THEN TO_NUMBER(PRINCIPALAMOUNT) 
            ELSE 0 
        END AS PRINCIPALAMOUNT
    FROM 
        imke.tmp_crb_accounts@edwdblnk
)
SELECT 
    NATIONALID,
    SUM(CASE WHEN EXTRACT(YEAR FROM DISBURSMENTDATE) = 2017 THEN PRINCIPALAMOUNT ELSE 0 END) AS Total_2017,
    SUM(CASE WHEN EXTRACT(YEAR FROM DISBURSMENTDATE) = 2018 THEN PRINCIPALAMOUNT ELSE 0 END) AS Total_2018,
    SUM(CASE WHEN EXTRACT(YEAR FROM DISBURSMENTDATE) = 2019 THEN PRINCIPALAMOUNT ELSE 0 END) AS Total_2019,
    SUM(CASE WHEN EXTRACT(YEAR FROM DISBURSMENTDATE) = 2020 THEN PRINCIPALAMOUNT ELSE 0 END) AS Total_2020,
    SUM(CASE WHEN EXTRACT(YEAR FROM DISBURSMENTDATE) = 2021 THEN PRINCIPALAMOUNT ELSE 0 END) AS Total_2021,
    SUM(CASE WHEN EXTRACT(YEAR FROM DISBURSMENTDATE) = 2022 THEN PRINCIPALAMOUNT ELSE 0 END) AS Total_2022
FROM 
    cte
GROUP BY 
    NATIONALID
"""
df = pd.read_sql_query(query1, connection)

In [11]:
query2 = """
WITH cte AS (
    SELECT 
        NATIONALID,
        PRODUCT,
        CASE 
            WHEN REGEXP_LIKE(PRINCIPALAMOUNT, '^\d+(\.\d+)?$') 
                THEN TO_NUMBER(PRINCIPALAMOUNT) 
            ELSE NULL 
        END AS PRINCIPALAMOUNT
    FROM 
        imke.tmp_crb_accounts@edwdblnk
)
SELECT 
    NATIONALID,
    PRODUCT,
    MAX(PRINCIPALAMOUNT) AS MAX_AMOUNT,
    MIN(PRINCIPALAMOUNT) AS MIN_AMOUNT,
    AVG(PRINCIPALAMOUNT) AS AVG_AMOUNT
FROM 
    cte
GROUP BY 
    NATIONALID,
    PRODUCT
"""

df2 = pd.read_sql_query(query2, connection)


/Users/asha.anab/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [12]:
query3 = """
WITH cte AS (
    SELECT 
        NATIONALID,
        CASE 
            WHEN PRODUCT LIKE '%Mobile%' OR 
                 PRODUCT LIKE '%Personal%' OR 
                 PRODUCT LIKE '%Business%' OR 
                 PRODUCT LIKE '%Credit%' OR 
                 PRODUCT LIKE '%Mortgage%' OR 
                 PRODUCT LIKE '%asset finance%' OR 
                 PRODUCT LIKE '%Trade finance%'
                THEN 1 
            ELSE 0 
        END AS PRODUCT_EXISTS,
        CASE 
            WHEN REGEXP_LIKE(PRINCIPALAMOUNT, '^\d+(\.\d+)?$') 
                THEN TO_NUMBER(PRINCIPALAMOUNT) 
            ELSE NULL 
        END AS PRINCIPALAMOUNT
    FROM 
        imke.tmp_crb_accounts@edwdblnk
)
SELECT 
    NATIONALID,
    MAX(PRODUCT_EXISTS) AS PRODUCT_EXISTS,
    MAX(PRINCIPALAMOUNT) AS MAX_AMOUNT_PRINCIPAL,
    MIN(PRINCIPALAMOUNT) AS MIN_AMOUNT_PRINCIPAL,
    AVG(PRINCIPALAMOUNT) AS AVG_AMOUNT_PRINCIPAL
FROM 
    cte
GROUP BY 
    NATIONALID
"""

df3 = pd.read_sql_query(query3, connection)


/Users/asha.anab/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
query4 = """
WITH cte AS (
    SELECT 
        NATIONALID,
        TO_NUMBER(
            CASE 
                WHEN WORSTDAYSINARREARS = 'NA' THEN '0'
                ELSE WORSTDAYSINARREARS
            END
        ) AS WORSTDAYSINARREARS,
        TO_DATE(LISTINGDATE, 'DD-MM-YYYY') AS LISTINGDATE
    FROM
        imke.tmp_crb_accounts@edwdblnk
),
date_to_check AS (
    SELECT
        TO_DATE('01-02-2022', 'DD-MM-YYYY') - 365 AS date_year_ago,
        TO_DATE('01-02-2022', 'DD-MM-YYYY') AS date_check
    FROM
        dual
)
SELECT
    NATIONALID,
    CASE
        WHEN SUM(CASE WHEN WORSTDAYSINARREARS BETWEEN 30 AND 59 
                        AND LISTINGDATE BETWEEN (SELECT date_year_ago FROM date_to_check) 
                                           AND (SELECT date_check FROM date_to_check)
                       THEN 1 
                       ELSE 0 
                  END) > 0 
         THEN 1
        ELSE 0
    END AS daysPastDueBetween30to59_in_12_months,
    CASE
        WHEN SUM(CASE WHEN WORSTDAYSINARREARS BETWEEN 60 AND 90 
                        AND LISTINGDATE BETWEEN (SELECT date_year_ago FROM date_to_check) 
                                           AND (SELECT date_check FROM date_to_check)
                       THEN 1 
                       ELSE 0 
                  END) > 0 
         THEN 1
        ELSE 0
    END AS dpd60_90_in_12_months,
    CASE
        WHEN SUM(CASE WHEN WORSTDAYSINARREARS > 90 
                        AND LISTINGDATE BETWEEN (SELECT date_year_ago FROM date_to_check) 
                                           AND (SELECT date_check FROM date_to_check)
                       THEN 1 
                       ELSE 0 
                  END) > 0 
         THEN 1
        ELSE 0
    END AS dpd_over_90_in_12_months
FROM
    cte
GROUP BY
    NATIONALID
"""

df4 = pd.read_sql_query(query4, connection)


/Users/asha.anab/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
query5 = """
SELECT 
    NATIONALID,
    SUM(CASE WHEN UPPER(LENDER) LIKE '%BANKS%' THEN 1 ELSE 0 END) AS BANKS_COUNT,
    SUM(CASE WHEN UPPER(LENDER) LIKE '%MICROFINANCE%' THEN 1 ELSE 0 END) AS MICROFINANCE_COUNT,
    SUM(CASE WHEN UPPER(LENDER) LIKE '%SACCO%' THEN 1 ELSE 0 END) AS SACCO_COUNT
FROM 
    imke.tmp_crb_accounts@edwdblnk
GROUP BY 
    NATIONALID
"""

df5 = pd.read_sql_query(query5, connection)


/Users/asha.anab/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
df.head(2)

,NATIONALID,TOTAL_2017,TOTAL_2018,TOTAL_2019,TOTAL_2020,TOTAL_2021,TOTAL_2022
0,20917572,127000.0,443659.0,819105.4,389780.0,393825.0,916237.0
1,24055763,120000.0,1960640.0,2278050.0,3045291.0,2099620.0,1322919.0


In [20]:
df2.head(2)

,NATIONALID,PRODUCT,MAX_AMOUNT,MIN_AMOUNT,AVG_AMOUNT
0,14463219,MOBILELOAN,90000.0,2000.0,38403.433923
1,29372599,MOBILELOAN,2500.0,2500.0,2500.000000


In [22]:
df3.head(2)

,NATIONALID,PRODUCT_EXISTS,MAX_AMOUNT_PRINCIPAL,MIN_AMOUNT_PRINCIPAL,AVG_AMOUNT_PRINCIPAL
0,20917572,0,55250.0,1067.0,21684.766667
1,22851450,0,1050000.0,1000.0,43993.538462


In [23]:
df4.head(2)

,NATIONALID,DAYSPASTDUEBETWEEN30TO59_IN_12_MONTHS,DPD60_90_IN_12_MONTHS,DPD_OVER_90_IN_12_MONTHS
0,20917572,0,0,0
1,24055763,0,0,1


In [24]:
df5.head(2)

,NATIONALID,BANKS_COUNT,MICROFINANCE_COUNT,SACCO_COUNT
0,20917572,0,0,0
1,22851450,0,0,0


In [26]:
pivot_df2 = pd.pivot_table(df2, values=['MIN_AMOUNT', 'MAX_AMOUNT', 'AVG_AMOUNT'], index='NATIONALID', columns='PRODUCT')

pivot_df2.columns = [f"{col[0]}_{col[1]}" for col in pivot_df2.columns]

pivot_df2.reset_index(inplace=True)


In [27]:
pivot_df2.head(2)

,NATIONALID,AVG_AMOUNT_ASSETFINANCELOANS,AVG_AMOUNT_BUSINESSEXPANSIONLOANS,AVG_AMOUNT_BUSINESSWORKINGCAPITALLOANS,AVG_AMOUNT_CREDITCARDS,AVG_AMOUNT_GROUPLOANS,AVG_AMOUNT_INSURANCEPREMIUMFINANCING,AVG_AMOUNT_MOBILELOAN,AVG_AMOUNT_MORTGAGELOANS,AVG_AMOUNT_OTHER,...,MIN_AMOUNT_CREDITCARDS,MIN_AMOUNT_GROUPLOANS,MIN_AMOUNT_INSURANCEPREMIUMFINANCING,MIN_AMOUNT_MOBILELOAN,MIN_AMOUNT_MORTGAGELOANS,MIN_AMOUNT_OTHER,MIN_AMOUNT_OVERDRAFT,MIN_AMOUNT_PERSONALLOANS,MIN_AMOUNT_TERMLOAN,MIN_AMOUNT_TRADEFINANCEFACILITIES
0,00206187,NaN,NaN,NaN,1000000.0,NaN,NaN,66748.800000,NaN,NaN,...,1000000.0,NaN,NaN,66748.8,NaN,NaN,NaN,NaN,NaN,NaN
1,00289191,NaN,NaN,NaN,110775.0,NaN,NaN,41656.521739,NaN,43755.0,...,50000.0,NaN,NaN,18000.0,NaN,20000.0,1341.21,NaN,NaN,NaN


In [28]:

merged_df = df.merge(df3, on='NATIONALID', how='left')
merged_df = merged_df.merge(df4, on='NATIONALID', how='left')
merged_df = merged_df.merge(df5, on='NATIONALID', how='left')
merged_df = merged_df.merge(pivot_df2, on='NATIONALID', how='left')

merged_df.head()


,NATIONALID,TOTAL_2017,TOTAL_2018,TOTAL_2019,TOTAL_2020,TOTAL_2021,TOTAL_2022,PRODUCT_EXISTS,MAX_AMOUNT_PRINCIPAL,MIN_AMOUNT_PRINCIPAL,...,MIN_AMOUNT_CREDITCARDS,MIN_AMOUNT_GROUPLOANS,MIN_AMOUNT_INSURANCEPREMIUMFINANCING,MIN_AMOUNT_MOBILELOAN,MIN_AMOUNT_MORTGAGELOANS,MIN_AMOUNT_OTHER,MIN_AMOUNT_OVERDRAFT,MIN_AMOUNT_PERSONALLOANS,MIN_AMOUNT_TERMLOAN,MIN_AMOUNT_TRADEFINANCEFACILITIES
0,20917572,127000.0,443659.0,819105.4,389780.00,393825.00,916237.00,0,55250.0,1067.00,...,NaN,NaN,NaN,3115.0,NaN,9500.0,1067.00,NaN,NaN,NaN
1,24055763,120000.0,1960640.0,2278050.0,3045291.00,2099620.00,1322919.00,0,1000000.0,631.00,...,NaN,NaN,NaN,1200.0,NaN,15000.0,631.00,NaN,NaN,NaN
2,1011051,0.0,500.0,5000.0,2500.00,300.00,7500.00,0,2500.0,300.00,...,NaN,NaN,NaN,500.0,NaN,500.0,300.00,NaN,NaN,NaN
3,22572816,0.0,0.0,3706.0,11100.00,27485.00,4298.00,0,20000.0,441.00,...,20000.0,NaN,NaN,500.0,NaN,NaN,441.00,NaN,NaN,NaN
4,25661450,157000.0,444800.0,294800.0,475499.64,137056.35,23903.96,0,20000.0,1144.95,...,NaN,NaN,NaN,3000.0,NaN,9400.0,1144.95,NaN,NaN,NaN
